In [5]:
from tools import solution_xls_extract_hfc_replacement as sxe  # The extraction pipeline custom made for HFCReplacement
from tools import create_expected_zip as cez
from tools import expected_ghost
from solution import factory
from pathlib import Path
import pandas as pd
import openpyxl
import importlib

In [6]:
excelfile = Path(r"C:\Users\Daniel\repos\RRS_HFCReplacement Model_June2020Integration.xlsm")
outdir = Path(r"C:\Users\Daniel\repos\solutions\solution\hfc_replacement")
outdir.mkdir(parents=True, exist_ok=True)

In [7]:
importlib.reload(sxe)

<module 'tools.solution_xls_extract_hfc_replacement' from 'C:\\Users\\Daniel\\repos\\solutions\\tools\\solution_xls_extract_hfc_replacement.py'>

In [4]:
# sxe.output_solution_python_file(outputdir=outdir, xl_filename=str(excelfile))

C:\Users\Daniel\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\reader\drawings.py:59: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)
C:\Users\Daniel\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
C:\Users\Daniel\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\Daniel\anaconda3\envs\pd-dev\lib\site-packages\openpyxl\reader\drawings.py:29: UserWarning: DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.
  warn("DrawingML support is incomplete and limited to charts and images only. Shapes and drawings will be lost.")
C:\Users\Daniel\repos\solutions\tools\solution_xls_extract_hfc_replacement.py:906: SettingWithCopyWarning: 
A value is trying to be set o

NOTE DMK: 07.08.21 extraction code runs.

I'll collect ISSUES here.
- ad is not extracted at all. They look very different from the others. They are in a sheet called "Adoption Calcs" in excel.

What WORKED?
- The two TAM scenarios and their mean were correctly converted to .csv from the TAM Data sheet in excel.
- Python ac in ScenarioRecord tab in excel according to docs.
- The three scenario names in ScenarioRecord are recognized correctly.

What is UNCLEAR?
- ca_pds_data looks good but I cannot find the numbers in the excel.

NOTES!
- VMA stands for Variable Meta Analysis. The result of VMA is a collection of scaler parameters of the model. There can be low, high or average versions of the parameter.
- RRS models deal with reduction and replacement. HFCReplacement is a RRS model and importantly models adoption as a proportion of the total addressable market (TAM).
- TAM stands for total addressable market. The unit of TAM is kton refrigerant.
- ODS stands for ozone-depleting substances.
- Two adoption scenarios and their average are in "Adoption Calcs" there is not "Adoption Data" sheet. The automated function looks for "Adoption Data"
- "Advanced Control" sheet is where researchers input settings and in Python those are translated to ac subdirectory.
- "Variable Meta-analysis-Open" sheet exists but there are barely any entries, only adoption data for Velders et al. (2017). The automated extraction function does not extract any of this data for unknown reasons.

solution_xls_extract.py FUNCTIONS!
* get_rrs_scenarios(wb, solution_category)
    - Extracts data from the ScenarioRecord sheet as it is stored in the ac subdirectory.
    - Needs fixing, most values in ac are 0.

How do models and scenarios actually work?
Consider the below code.

In [6]:
(constructor,scenarios) = factory.one_solution_scenarios("hfc_replacement")
myscenario = constructor(scenarios[0])
bi_co2e = myscenario.c2.co2eq_direct_reduced_emissions()

C:\Users\Daniel\repos\solutions\model\unitadoption.py:627: RuntimeWarning: invalid value encountered in subtract
  growth_array = np.max([growth.values[1:] - growth.values[:-1], np.full(growth.values[1:].shape, 0.)], axis=0)


The `factory` module is imported from the solution package. The `one_solution_scenarios` function then imports a single solution that is specified by a string passed to it. It returns the `Scenario` class that is defined in the `__init__.py` script of the loaded solution. The `__init__` is central to the functionality of the solution. Any kind of data and computation that is part of the solution can be found in `__init__.py` but it relies on functionality that is implemented in the `model` package.


Consider two important functionalities: unit adoption and co2 calculations. The model package contains the `co2calcs` and the `unitadoption` modules and `__init__.py` imports both. To use these modules, instances of the core classes they define must be created. `__init__.py` attaches an instance of `unitadoption.UnitAdoption` to `self.ua`. To create this instance it needs to pass data to it. The way this data flows between different parts of the solution is central to understanding it. `UnitAdoption` for example depends on `HelperTables`. 

In [24]:
myscenario.ht.pds_adoption_data_per_region  # Place of Python excel divergence in Year 2014
myscenario.ac.soln_pds_adoption_basis  
# 'Fully Customized PDS'
# This means pds_adoption_data_per_region comes from pds_ca
myscenario.pds_ca.adoption_data_per_region()  # Already diverged here
# pds_ca is an instance of CustomAdoption
# Divergence resolved

# Next divergence in:
myscenario.ht.soln_ref_funits_adopted()
myscenario.ac.soln_ref_adoption_basis
# 'Default'
# myscenario.ht.soln_ref_funits_adopted() is interpolated linearly from ref_datapoints
# The divergence occurs because ht_ref_datapoints.loc[2050] is incorrect. Hardcoding it to the 2050 value from the excel
# sheet 'Helper Tables' fixes it.

,World,OECD90,Eastern Europe,Asia (Sans Japan),Middle East and Africa,Latin America,China,India,EU,USA
Year,,,,,,,,,,
2014,1.153458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015,1.562594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016,1.971729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017,2.380865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018,2.790000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019,3.199135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020,3.608271,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021,4.017406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022,4.426542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Create the expected zip

In [6]:
csvdirectory = Path(r"C:\Users\Daniel\repos\pd_temp")
cez.create_expected_zip(csvdirectory)

In [8]:
outdir = r"C:\Users\Daniel\repos\solutions\solution\hfc_replacement"
#!python -m pytest -s $outdir
!python -m pytest $outdir -s

============================= test session starts =============================
platform win32 -- Python 3.9.6, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Git: v0.13.0-42-g1c975591 ()
rootdir: C:\Users\Daniel\repos\solutions, configfile: tox.ini

error: unknown option `show-current'
usage: git branch [<options>] [-r | -a] [--merged | --no-merged]
   or: git branch [<options>] [-l] [-f] <branch-name> [<start-point>]
   or: git branch [<options>] [-r] (-d | -D) <branch-name>...
   or: git branch [<options>] (-m | -M) [<old-branch>] <new-branch>
   or: git branch [<options>] (-c | -C) [<old-branch>] <new-branch>
   or: git branch [<options>] [-r | -a] [--points-at]
   or: git branch [<options>] [-r | -a] [--format]

Generic options
    -v, --verbose         show hash and subject, give twice for upstream branch
    -q, --quiet           suppress informational messages
    -t, --track           set up tracking mode (see git-pull(1))
    -u, --set-upstream-to <upstream>
                          change the upstream info
    --unset-upstream      Unset the upstream info
    --color[=<when>]      use colored output
    -r, --remotes         act on remote-tracking branches
    --contains <commit>   print only branches that contain the c


collected 2 items

solution\hfc_replacement\tests\test_hfc_replacement.py .Checking scenario 0: PDS1-67p2050-LowerLowGWP Adoption
**** Skipped scenario 1 'PDS2-82p2050-Median'
**** Skipped scenario 2 'PDS3-97p2050-Upper'
.

============================== warnings summary ===============================
solution/hfc_replacement/tests/test_hfc_replacement.py::test_hfc_replacement_loader
solution/hfc_replacement/tests/test_hfc_replacement.py::test_hfc_replacement_results
  C:\Users\Daniel\repos\solutions\model\unitadoption.py:627: RuntimeWarning: invalid value encountered in subtract
    growth_array = np.max([growth.values[1:] - growth.values[:-1], np.full(growth.values[1:].shape, 0.)], axis=0)

-- Docs: https://docs.pytest.org/en/stable/warnings.html
======================== 2 passed, 2 warnings in 6.86s ========================


In [4]:
outdir = r"C:\Users\Daniel\repos\solutions\solution\hfc_replacement"
#!python -m pytest -s $outdir
!python -m pytest $outdir -s

============================= test session starts =============================
platform win32 -- Python 3.9.6, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
Git: v0.13.0-38-g77adae09 ()
rootdir: C:\Users\Daniel\repos\solutions, configfile: tox.ini
collected 2 items

solution\hfc_replacement\tests\test_hfc_replacement.py .Checking scenario 0: PDS1-67p2050-LowerLowGWP Adoption
**** Skipped scenario 1 'PDS2-82p2050-Median'
**** Skipped scenario 2 'PDS3-97p2050-Upper'
.

============================== warnings summary ===============================
solution/hfc_replacement/tests/test_hfc_replacement.py::test_hfc_replacement_loader
solution/hfc_replacement/tests/test_hfc_replacement.py::test_hfc_replacement_results
  C:\Users\Daniel\repos\solutions\model\unitadoption.py:627: RuntimeWarning: invalid value encountered in subtract
    growth_array = np.max([growth.values[1:] - growth.values[:-1], np.full(growth.values[1:].shape, 0.)], axis=0)

-- Docs: https://docs.pytest.org/en/stable/warnings.htm

error: unknown option `show-current'
usage: git branch [<options>] [-r | -a] [--merged | --no-merged]
   or: git branch [<options>] [-l] [-f] <branch-name> [<start-point>]
   or: git branch [<options>] [-r] (-d | -D) <branch-name>...
   or: git branch [<options>] (-m | -M) [<old-branch>] <new-branch>
   or: git branch [<options>] (-c | -C) [<old-branch>] <new-branch>
   or: git branch [<options>] [-r | -a] [--points-at]
   or: git branch [<options>] [-r | -a] [--format]

Generic options
    -v, --verbose         show hash and subject, give twice for upstream branch
    -q, --quiet           suppress informational messages
    -t, --track           set up tracking mode (see git-pull(1))
    -u, --set-upstream-to <upstream>
                          change the upstream info
    --unset-upstream      Unset the upstream info
    --color[=<when>]      use colored output
    -r, --remotes         act on remote-tracking branches
    --contains <commit>   print only branches that contain the c

In [11]:
import os

os.path.join(scenarios[0], 'TAM Data')

'PDS1-67p2050-LowerLowGWP Adoption\\TAM Data'

In [8]:
not myscenario.name == 'Refrigerant Management - HFC Replacement'

False

In [9]:
dir(myscenario.ac)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_init__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_hash_item',
 '_substitute_vma',
 'avoided_deforest_with_intensification',
 'carbon_not_emitted_after_harvesting',
 'ch4_co2_per_funit',
 'ch4_is_co2eq',
 'co2eq_conversion_source',
 'conv_2014_cost',
 'conv_annual_energy_used',
 'conv_avg_annual_use',
 'conv_emissions_per_funit',
 'conv_expected_lifetime',
 'conv_first_cost_efficiency_rate',
 'conv_first_cost_learning_rate',
 'conv_fixed_oper_cost_per_iunit',
 'conv_fuel_consumed_per_funit',
 'conv_fuel_cost_per_funit',
 'conv_fuel_emissions_factor',
 'conv_indirect_co2_is_iunits',
 

In [1]:
myscenario.tm.ref_tam_per_region()

NameError: name 'myscenario' is not defined